# Is the SAT unfair?  Cleaning NYC high school data to start finding the answer  

### Goal of this project

The goal of this project is to clean and consolidate several datasets relating to the SAT (Scholastic Aptitude Test) and NYC (New York City) high schools.  The final cleaned dataset will be used in separate projects to explore if the SAT is unfair to particular groups of students.  The annual SAT is used by U.S. colleges and universities to determine which students to admit into their programs.   

### Approach

We will begin by exploring each of the datasets to understand the information they contain and how they can be consolidated.  We will then clean the datasets to make them ready for further analysis and modeling (i.e. correcting data types and errors, removing unnecessary characters and columns, filling in missing values etc...).  Finally, we will consolidate all of the data into a single cleaned dataset and export it to a csv (comma separated value) file that is ready to use for further exploratory data analysis, visualization and predictive modeling.


### Summary of results

The raw data consists of 8 separate datasets containing missing values, incorrect data types, data errors, unnecessary columns and a combined total of 65,875 rows and 3,861 columns.  The final result of this project is a single consolidated csv file containing 362 rows and 79 columns.  This final dataset is clean, consistent, relevant and ready to use for further analysis and modeling to explore the fairness of the SAT.

#  Introduction to the data

New York City has published data on student SAT scores by high school, along with additional related datasets, which can all be found [here](https://data.cityofnewyork.us/browse?category=Education).  The following is a list of the datasets we will be cleaning and consolidating along with a brief description of each.  Each one links to the source page where the data can be downloaded:  

* **[SAT scores by school](https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4)**  (`"sat_results.csv"`):  NYC SAT results by school for the graduating seniors of 2012.  At the time of this writing, this was the most recent SAT data available.
* **[High school directory](https://data.cityofnewyork.us/Education/2014-2015-DOE-High-School-Directory/n3p6-zve2)** (`"hs_directory.csv"`): Various descriptive information for each high school in NYC for the school year 2014-2015.  We are using this school year because it is the earliest this data was provided in csv format.  We are assuming the descriptive information for most schools (i.e. location) will not have changed much in just a few years.
* **[Class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)** (`"class_size.csv"`): NYC average class sizes by school for 2010-2011.  We are using this school year because this data was not provided "by school" for 2011-2012.
* **[AP test results](https://data.cityofnewyork.us/Education/2012-AP-Results/9ct9-prf9)** (`"ap_results.csv"`): NYC AP (Advanced Placement) exam results for 2012.  Passing an optional AP exam in a particular subject can earn a student college credit in that subject.  
* **[Graduation outcomes](https://data.cityofnewyork.us/Education/2005-2010-Graduation-Outcomes-School-Level/vh2h-md7a)** (`"graduation.csv"`): Various NYC graduation data for graduating classes 2005-2010.  These were the latest graduating years this data was available for.
* **[Demographics](https://data.cityofnewyork.us/Education/2006-2012-School-Demographics-and-Accountability-S/ihfw-zy9j)** (`"demographics.csv"`): 2006-2012 NYC public school student populations by grade, special programs, ethnicity, gender etc...  
* **[School survey](https://data.cityofnewyork.us/Education/2010-2011-NYC-School-Survey/mnz3-dyi8)** (`"survey_all.txt"`, `"survey_d75.txt"`): 2011 parent, teacher and student survey results for each school.  Survey questions assess each group's opinions on academic expectations, communication, engagement, safety and respect etc...  We are using the 2011 data because it was not available for 2012.


### Reading in the csv data 

First, we will read each csv data file into a Pandas Dataframe and then store them in a dictionary named `"data"`.  Using a dictionary provides efficient access to the different datasets by storing them all in one data structure. 

In [1]:
import pandas as pd

# Set the display options, read each csv file into a dataframe, store 
# them in a dictionary and view the "key" values of the dictionary.
pd.options.display.max_columns = 1000  
pd.options.display.max_rows = 1000
data_files = ["ap_results.csv", "class_size.csv", "demographics.csv", 
              "graduation.csv", "hs_directory.csv", "sat_results.csv"]
data = {}
for file in data_files:
    dataframe = pd.read_csv("../data/raw/{0}".format(file))
    key = file.replace(".csv", "")
    data[key] = dataframe
data.keys()

dict_keys(['ap_results', 'class_size', 'demographics', 'graduation', 'hs_directory', 'sat_results'])

# Understanding the data  

We will look at the first 5 rows of each of the dataframes to see what we can learn on initial inspection. 


### Understanding the `"sat_results"` dataframe

We are interested in exploring relationships between the SAT data and the other datasets. 

Referencing `"SAT_Results_Data_Dictionary.xlsx"` in the `"references"` folder of this project, each row provides the average SAT scores for each high school in NYC.  Each high school has a unique identification code called a `"DBN"` (district, borough, school number).  The scores are broken down into the 3 main sections of the test: Critical Reading, Math and Writing.  

We can explore correlations between this data and the other dataframes to uncover any bias in the SAT.

In [2]:
# View the first 5 rows of the "sat_results" dataframe.
data["sat_results"].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


### Understanding the `"ap_results"` dataframe  

Referencing `"AP_Results_Data_Dictionary.xlsx"` in the `"references"` folder of this project, this dataset provides AP test results data for each NYC high school.  We can use this data to find any correlation between SAT results and AP results.  We can use the `"DBN"` column to join this data with the other dataframes.

In [3]:
# View the first 5 rows of the "ap_results" dataframe.
data["ap_results"].head()

,DBN,SCHOOL NAME,Num of AP Test Takers,Num of AP Total Exams Taken,Num of AP Exams Passed
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,s,s,s
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,37,53,21
2,01M450,EAST SIDE COMMUNITY SCHOOL,12,12,s
3,01M458,FORSYTH SATELLITE ACADEMY,s,s,s
4,01M509,MARTA VALLE HIGH SCHOOL,14,15,s


### Understanding the `"class_size"` dataframe  

Referencing `"Class_Size_Data_Dictionary.xlsx"` in the `"references"` folder of this project, this dataset provides average class sizes for each school, by grade and program type for grades 5-9 and 9-12, aggregated by program type and core course.  We can use this data to find any correlation between SAT results and class size.

Each row represents a specific `"GRADE"` and `"PROGRAM TYPE"` for that school.  We will have to filter for the rows that contain the `"GRADE"` and `"PROGRAM TYPE"` values that are most suitable for our analysis.  

There is also no `"DBN"` (district, borough, school number) column in this dataframe.  However, according to `"Class_Size_Data_Dictionary.xlsx"`, the `"CSD"` column represents the district and the `"SCHOOL CODE"` column represents the borough and school number.  This means we can create a `"DBN"` column by combining the `"CSD"` and `"SCHOOL CODE"` columns.  If `"CSD"` is only a single digit, we need to add a leading `"0"` to make the `"DBN"` consistent with the other dataframes.  This will allow us to join this data with the other dataframes.

In [4]:
# View the first 5 rows of the "class_size" dataframe.
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


### Understanding the `"demographics"` dataframe  

Referencing `"Demographics_Data_Dictionary.xlsx"` in the `"references"` folder of this project, this dataset provides various demographic data for each NYC high school.  Some examples include: `"ell_percent"` (percentage of English language learners), `"asian_per"` (percentage of Asian students) and `"female_per"` (percentage of female students).  This data will be instrumental in exploring if the SAT is unfair towards any particular demographic groups.

Each row represents a specific school year for that school.  We will have to filter for the rows that contain the school years that are most suitable for our analysis.  We can use the `"DBN"` column to join this data with the other dataframes.

In [5]:
# View the first 5 rows of the "demographics" dataframe.
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,38,52,29,38,NaN,NaN,NaN,NaN,NaN,NaN,36.0,12.8,57.0,20.3,25,9,10,3.6,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,34,42,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,15.6,55.0,22.6,19,15,18,7.4,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,38,47,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,19.9,60.0,23.0,20,14,16,6.1,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,34,39,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,19.0,62.0,24.6,21,17,16,6.3,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,30,24,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,19.2,46.0,22.1,14,14,16,7.7,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4


### Understanding the `"graduation"` dataframe  

Referencing `"Graduation_Data_Dictionary.xlsx"` in the `"references"` folder of this project, this dataset provides various graduation statistics for each NYC high school.  Some examples include: `"Total Grads - % of cohort"` (percentage of students that graduated) and `"Dropped Out - % of Cohort"` (percentage of students that dropped out).  We can use this data to find any correlation between SAT results and graduation rates.

Each row represents a specific `"Demographic"` (i.e. Total Cohort, White, Black etc...) and `"Cohort"` (the school year the SAT was administered) for that school.  We will have to filter for the rows that contain the `"Demographic"` and `"Cohort"` values that are most suitable for our analysis.  We can use the `"DBN"` column to join this data with the other dataframes.

In [6]:
# View the first 5 rows of the "graduation" dataframe.
data["graduation"].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3%,17,30.9%,45.9%,0,0%,0%,17,30.9%,45.9%,20,36.4%,54.1%,15,27.3%,3,5.5%
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2%,27,42.2%,62.8%,0,0%,0%,27,42.2%,62.8%,16,25%,37.200000000000003%,9,14.1%,9,14.1%
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4%,37,47.4%,84.1%,0,0%,0%,37,47.4%,84.1%,7,9%,15.9%,15,19.2%,11,14.1%


### Understanding the `"hs_directory"` dataframe  

Referencing `"HS_Directory_Data_Dictionary.xlsx"` in the `"references"` folder of this project, this dataset provides various descriptive information for each NYC high school.  Some examples include: `"boro"` (borough where the school is located) and `"Location 1"` (address and geographic coordinates).  We can use `"Location 1"` to visualize the SAT results on a map of NYC.  This will let us see any geographic patterns in SAT results.

Each row represents descriptive information for each NYC high school.  We can use the `"dbn"` column to join this data with the other dataframes.

In [7]:
# View the first 5 rows of the "hs_directory" dataframe.
data["hs_directory"].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,"B41, B43, B44-SBS, B45, B48, B49, B69","2, 3, 4, 5, F, S to Botanic Garden ; B, Q to P...",883 Classon Avenue,Brooklyn,NY,11225,Bkmusicntheatre.com,399.0,Prospect Heights Educational Campus,NaN,Brooklyn School for Music & Theatre (BSMT) use...,We offer highly competitive positions in our D...,Spanish,"English Language and Composition, United State...",NaN,NaN,"Variety of clubs: Chess, The Step Team, Fashio...","Baseball, Basketball & JV Basketball, Cross Co...","Basketball, Cross Country, Indoor Track, Outdo...",NaN,NaN,F.Y.R.EZONE (Finding Your Rhythm thru Educatio...,NaN,NaN,"In 2002, Roundabout Theatre was selected by Ne...",One To World‘s Global Classroom connects New Y...,NaN,NaN,NaN,NaN,NaN,8:10 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Brooklyn students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67..."
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,"Bx13, Bx15, Bx17, Bx21, Bx35, Bx4, Bx41, Bx4A,...","2, 5 to Intervale Ave",1110 Boston Road,Bronx,NY,10456,www.hsvd.org,378.0,Morris Educational Campus,NaN,The High School for Violin and Dance (HSVD) is...,Freshmen take both violin and dance; College N...,Spanish,NaN,NaN,NaN,Advancement via Individual Determination (AVID...,"Baseball, Basketball & JV Basketball, Volleyball","Basketball, Softball, Volleyball",NaN,Morris Educational Campus Basketball and Volle...,McGraw Hill - Big Brother Big Sister,NaN,"Hostos Community College, Monroe College, Teac...",Bronx Arts Ensemble,buildOn,Print International,NaN,Bronx Cares,Our students are required to take four years o...,"Student Summer Orientation, Summer Internship ...",8:00 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Bronx students or residents who at...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026..."
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,"Bx1, Bx11, Bx13, Bx18, Bx2, Bx3, Bx32, Bx35, Bx36","4 to Mt Eden Ave ; B, D to 170th St",1501 Jerome Avenue,Bronx,NY,10452,http://schools.nyc.gov/schoolportals/09/X327,543.0,DOE New Settlement Community Campus,NaN,At the Comprehensive Model School Project (CMS...,"After-school and Saturday Tutoring, Advisory, ...",Spanish,"Biology, Chemistry, United States History",NaN,NaN,"Choir, Gaming, Girls Club, Newspaper, Spanish,...",NaN,NaN,NaN,"As we expand, we plan to offer PSAL sports.",New Settlement Community Center,Montefiore Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Dress Code Required: white or baby blue button...,NaN,8:00 AM,4:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to continuing 8th graders,Then to Bronx students or residents who attend...,Then to New York City residents who attend an ...,Then to Bronx stud

# Reading in, understanding and combining the survey data  

Referencing `"Survey_Data_Dictionary.xls"` in the `"references"` folder of this project, the `"survey_all.txt"` and `"survey_d75.txt"` files are datasets summarizing the results of the 2011 NYC School Survey.  This is an annual survey that asks parents, teachers and students their opinions on academic expectations, communication, engagement, safety, respect and various other topics at their respective schools.  


### Reading in and understanding the `"survey_all.txt"` data  

The `"survey_all.txt"` data file provides the survey results for all community schools.  We will read it into a dataframe named `"all_survey"`.  

Some of the columns include: `"saf_s_11"` (safety and respect score based on student responses) and `"aca_t_11"` (academic expectations score based on teacher responses).  We can use this data to find any correlation between SAT results and parents', teachers' and students' opinions about their respective schools.

Each row represents the survey results for each NYC high school.  We can use the `"dbn"` column to join this data with the other dataframes.

In [8]:
# Read "survey_all.txt" into a dataframe and view the first 5 rows.
all_survey = pd.read_csv('../data/raw/survey_all.txt', delimiter='\t',
                         encoding='windows-1252')
all_survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q2h,p_q7a,p_q7b,p_q7c,p_q7d,p_q8a,p_q8b,p_q8c,p_q8d,p_q8e,p_q8f,p_q2b,p_q2d,p_q2e,p_q2f,p_q2g,p_q3a,p_q3b,p_q4b,p_q4c,p_q11c,p_q2a,p_q2c,p_q3c,p_q6a,p_q6b,p_q11d,p_q11e,p_q5,p_q4a,p_q4d,p_q4e,p_q11a,p_q11b,p_q11f,p_q1,p_q3d,p_q9,p_q10,p_q12aa,p_q12ab,p_q12ac,p_q12ad,p_q12ba,p_q12bb,p_q12bc,p_q12bd,p_q1_1,p_q1_2,p_q1_3,p_q1_4,p_q1_5,p_q1_6,p_q1_7,p_q1_8,p_q1_9,p_q1_10,p_q1_11,p_q1_12,p_q1_13,p_q1_14,p_q2a_1,p_q2a_2,p_q2a_3,p_q2a_4,p_q2a_5,p_q2b_1,p_q2b_2,p_q2b_3,p_q2b_4,p_q2b_5,p_q2c_1,p_q2c_2,p_q2c_3,p_q2c_4,p_q2c_5,p_q2d_1,p_q2d_2,p_q2d_3,p_q2d_4,p_q2d_5,p_q2e_1,p_q2e_2,p_q2e_3,p_q2e_4,p_q2e_5,p_q2f_1,p_q2f_2,p_q2f_3,p_q2f_4,p_q2f_5,p_q2g_1,p_q2g_2,p_q2g_3,p_q2g_4,p_q2g_5,p_q2h_1,p_q2h_2,p_q2h_3,p_q2h_4,p_q2h_5,p_q3a_1,p_q3a_2,p_q3a_3,p_q3a_4,p_q3a_5,p_q3b_1,p_q3b_2,p_q3b_3,p_q3b_4,p_q3b_5,p_q3c_1,p_q3c_2,p_q3c_3,p_q3c_4,p_q3c_5,p_q3d_1,p_q3d_2,p_q3d_3,p_q3d_4,p_q3d_5,p_q4a_1,p_q4a_2,p_q4a_3,p_q4a_4,p_q4a_5,p_q4b_1,p_q4b_2,p_q4b_3,p_q4b_4,p_q4b_5,p_q4c_1,p_q4c_2,p_q4c_3,p_q4c_4,p_q4c_5,p_q4d_1,p_q4d_2,p_q4d_3,p_q4d_4,p_q4d_5,p_q4e_1,p_q4e_2,p_q4e_3,p_q4e_4,p_q4e_5,p_q5a,p_q5b,p_q5c,p_q5d,p_q5e,p_q5f,p_q5g,p_q5h,p_q5i,p_q5j,p_q6a_1,p_q6a_2,p_q6a_3,p_q6a_4,p_q6a_5,p_q6b_1,p_q6b_2,p_q6b_3,p_q6b_4,p_q6b_5,p_q7a_1,p_q7a_2,p_q7a_3,p_q7a_4,p_q7a_5,p_q7b_1,p_q7b_2,p_q7b_3,p_q7b_4,p_q7b_5,p_q7c_1,p_q7c_2,p_q7c_3,p_q7c_4,p_q7c_5,p_q7d_1,p_q7d_2,p_q7d_3,p_q7d_4,p_q7d_5,p_q8a_1,p_q8a_2,p_q8a_3,p_q8a_4,p_q8a_5,p_q8b_1,p_q8b_2,p_q8b_3,p_q8b_4,p_q8b_5,p_q8c_1,p_q8c_2,p_q8c_3,p_q8c_4,p_q8c_5,p_q8d_1,p_q8d_2,p_q8d_3,p_q8d_4,p_q8d_5,p_q8e_1,p_q8e_2,p_q8e_3,p_q8e_4,p_q8e_5,p_q8f_1,p_q8f_2,p_q8f_3,p_q8f_4,p_q8f_5,p_q9_1,p_q9_2,p_q9_3,p_q9_4,p_q9_5,p_q9_6,p_q9_7,p_q9_8,p_q9_9,p_q9_10,p_q10a,p_q10b,p_q10c,p_q10d,p_q10e,p_q10f,p_q10g,p_q10h,p_q10i,p_q10j,p_q10k,p_q10l,p_q11a_1,p_q11a_2,p_q11a_3,p_q11a_4,p_q11b_1,p_q11b_2,p_q11b_3,p_q11b_4,p_q11c_1,p_q11c_2,p_q11c_3,p_q11c_4,p_q11d_1,p_q11d_2,p_q11d_3,p_q11d_4,p_q11e_1,p_q11e_2,p_q11e_3,p_q11e_4,p_q11f_1,p_q11f_2,p_q11f_3,p_q11f_4,p_q12aa_1,p_q12aa_2,p_q12aa_3,p_q12aa_4,p_q12aa_5,p_q12ab_1,p_q12ab_2,p_q12ab_3,p_q12ab_4,p_q12ab_5,p_q12ac_1,p_q12ac_2,p_q12ac_3,p_q12ac_4,p_q12ac_5,p_q12ad_1,p_q12ad_2,p_q12ad_3,p_q12ad_4,p_q12ad_5,p_q12ba_1,p_q12ba_2,p_q12ba_3,p_q12ba_4,p_q12ba_5,p_q12bb_1,p_q12bb_2,p_q12bb_3,p_q12bb_4,p_q12bb_5,p_q12bc_1,p_q12bc_2,p_q12bc_3,p_q12bc_4,p_q12bc_5,p_q12bd_1,p_q12bd_2,p_q12bd_3,p_q12bd_4,p_q12bd_5,p_N_q1_1,p_N_q1_2,p_N_q1_3,p_N_q1_4,p_N_q1_5,p_N_q1_6,p_N_q1_7,p_N_q1_8,p_N_q1_9,p_N_q1_10,p_N_q1_11,p_N_q1_12,p_N_q1_13,p_N_q1_14,p_N_q2a_1,p_N_q2a_2,p_N_q2a_3,p_N_q2a_4,p_N_q2a_5,p_N_q2b_1,p_N_q2b_2,p_N_q2b_3,p_N_q2b_4,p_N_q2b_5,p_N_q2c_1,p_N_q2c_2,p_N_q2c_3,p_N_q2c_4,p_N_q2c_5,p_N_q2d_1,p_N_q2d_2,p_N_q2d_3,p_N_q2d_4,p_N_q2d_5,p_N_q2e_1,p_N_q2e_2,p_N_q2e_3,p_N_q2e_4,p_N_q2e_5,p_N_q2f_1,p_N_q2f_2,p_N_q2f_3,p_N_q2f_4,p_N_q2f_5,p_N_q2g_1,p_N_q2g_2,p_N_q2g_3,p_N_q2g_4,p_N_q2g_5,p_N_q2h_1,p_N_q2h_2,p_N_q2h_3,p_N_q2h_4,p_N_q2h_5,p_N_q3a_1,p_N_q3a_2,p_N_q3a_3,p_N_q3a_4,p_N_q3a_5,p_N_q3b_1,p_N_q3b_2,p_N_q3b_3,p_N_q3b_4,p_N_q3b_5,p_N_q3c_1,p_N_q3c_2,p_N_q3c_3,p_N_q3c_4,p_N_q3c_5,p_N_q3d_1,p_N_q3d_2,p_N_q3d_3,p_N_q3d_4,p_N_q3d_5,p_N_q4a_1,p_N_q4a_2,p_N_q4a_3,p_N_q4a_4,p_N_q4a_5,p_N_q4b_1,p_N_q4b_2,p_N_q4b_3,p_N_q4b_4,p_N_q4b_5,p_N_q4c_1,p_N_q4c_2,p_N_q4c_3,p_N_q4c_4,p_N_q4c_5,p_N_q4d_1,p_N_q4d_2,p_N_q4d_3,p_N_q4d_4,p_N_q4d_5,p_N_q4e_1,p_N_q4e_2,p_N_q4e_3,p_N_q4e_4,p_N_q4e_5,p_N_q5a,p_N_q5b,p_N_q5c,p_N_q5d,p_N_q5e,p_N_q5f,p_N_q5g,p_N_q5h,p_N_q5i,p_N_q5j,p_N_q6a_1,p_N_q6a_2,p_N_q6a_3,p_N_q6a_4,p_N_q6a_5,p_N_q6b_1,p_N_q6b_2,p_N_q6b_3,p_N_q6b_4,p_N_q6b_5,p_N_q7a_1,p_N_q7a_2,p_N_q7a_3,p_N_q7a_4,p_N_q7a_5,p_N_q7b_1,p_N_q7b_2,p_N_q7b_3,p_N_q7b_4,p_N_q7b_5,p_N_q7c_1,p_N_q7c_2,p_N_q7c_3,p_N_q7c_4,p_N_q7c_5,p_N_q7d_1,p_N_q7d_2,p_N_q7d_3,p_N_q7d_

### Reading in and understanding the `"survey_d75.txt"` data  

The `"survey_d75.txt"` data file provides the survey results for all District 75 schools.  We will read it into a dataframe named `"d75_survey"`.  

The structure of the dataframe is essentially the same as `"all_survey"`.  Although there are less total columns, the primary ones concerning parents', teachers' and students' opinions about school engagement, safety, academic expectations etc... remain common in both dataframes.

In [9]:
# Read "survey_d75.txt" into a dataframe and view the first 5 rows.
d75_survey = pd.read_csv('../data/raw/survey_d75.txt', delimiter='\t',
                         encoding='windows-1252')
d75_survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q1c,p_q10a,p_q10b,p_q10c,p_q10d,p_q10e,p_q10f,p_q11a,p_q11b,p_q11c,p_q11d,p_q11e,p_q1b,p_q1e,p_q1f,p_q2a,p_q2b,p_q3c,p_q3d,p_q4a,p_q6c,p_q12c,p_q1a,p_q1d,p_q3a,p_q3b,p_q3e,p_q4b,p_q4c,p_q6e,p_q7,p_q8a,p_q8b,p_q12d,p_q1g,p_q6a,p_q6b,p_q6d,p_q6f,p_q6g,p_q6h,p_q12a,p_q12b,p_q12e,p_q12f,p_q5,p_q9,p_q13a,p_q13b,p_q13c,p_q13d,p_q14a,p_q14b,p_q14c,p_q14d,p_q1a_1,p_q1a_2,p_q1a_3,p_q1a_4,p_q1a_5,p_q1b_1,p_q1b_2,p_q1b_3,p_q1b_4,p_q1b_5,p_q1c_1,p_q1c_2,p_q1c_3,p_q1c_4,p_q1c_5,p_q1d_1,p_q1d_2,p_q1d_3,p_q1d_4,p_q1d_5,p_q1e_1,p_q1e_2,p_q1e_3,p_q1e_4,p_q1e_5,p_q1f_1,p_q1f_2,p_q1f_3,p_q1f_4,p_q1f_5,p_q1g_1,p_q1g_2,p_q1g_3,p_q1g_4,p_q1g_5,p_q2a_1,p_q2a_2,p_q2a_3,p_q2a_4,p_q2a_5,p_q2b_1,p_q2b_2,p_q2b_3,p_q2b_4,p_q2b_5,p_q3a_1,p_q3a_2,p_q3a_3,p_q3a_4,p_q3a_5,p_q3b_1,p_q3b_2,p_q3b_3,p_q3b_4,p_q3b_5,p_q3c_1,p_q3c_2,p_q3c_3,p_q3c_4,p_q3c_5,p_q3d_1,p_q3d_2,p_q3d_3,p_q3d_4,p_q3d_5,p_q3e_1,p_q3e_2,p_q3e_3,p_q3e_4,p_q3e_5,p_q4a_1,p_q4a_2,p_q4a_3,p_q4a_4,p_q4a_5,p_q4b_1,p_q4b_2,p_q4b_3,p_q4b_4,p_q4b_5,p_q4c_1,p_q4c_2,p_q4c_3,p_q4c_4,p_q4c_5,p_q5a,p_q5b,p_q5c,p_q5d,p_q5e,p_q5f,p_q5g,p_q5h,p_q5i,p_q5j,p_q5k,p_q5l,p_q5m,p_q5n,p_q6a_1,p_q6a_2,p_q6a_3,p_q6a_4,p_q6a_5,p_q6b_1,p_q6b_2,p_q6b_3,p_q6b_4,p_q6b_5,p_q6c_1,p_q6c_2,p_q6c_3,p_q6c_4,p_q6c_5,p_q6d_1,p_q6d_2,p_q6d_3,p_q6d_4,p_q6d_5,p_q6e_1,p_q6e_2,p_q6e_3,p_q6e_4,p_q6e_5,p_q6f_1,p_q6f_2,p_q6f_3,p_q6f_4,p_q6f_5,p_q6g_1,p_q6g_2,p_q6g_3,p_q6g_4,p_q6g_5,p_q6h_1,p_q6h_2,p_q6h_3,p_q6h_4,p_q6h_5,p_q7a,p_q7b,p_q7c,p_q7d,p_q7e,p_q7f,p_q7g,p_q7h,p_q7i,p_q7j,p_q7k,p_q7l,p_q7m,p_q7n,p_q8a_1,p_q8a_2,p_q8a_3,p_q8a_4,p_q8a_5,p_q8b_1,p_q8b_2,p_q8b_3,p_q8b_4,p_q8b_5,p_q9_1,p_q9_2,p_q9_3,p_q9_4,p_q9_5,p_q9_6,p_q9_7,p_q9_8,p_q9_9,p_q9_10,p_q9_11,p_q9_12,p_q9_13,p_q9_14,p_q9_15,p_q10a_1,p_q10a_2,p_q10a_3,p_q10a_4,p_q10a_5,p_q10b_1,p_q10b_2,p_q10b_3,p_q10b_4,p_q10b_5,p_q10c_1,p_q10c_2,p_q10c_3,p_q10c_4,p_q10c_5,p_q10d_1,p_q10d_2,p_q10d_3,p_q10d_4,p_q10d_5,p_q10e_1,p_q10e_2,p_q10e_3,p_q10e_4,p_q10e_5,p_q10f_1,p_q10f_2,p_q10f_3,p_q10f_4,p_q10f_5,p_q11a_1,p_q11a_2,p_q11a_3,p_q11a_4,p_q11a_5,p_q11b_1,p_q11b_2,p_q11b_3,p_q11b_4,p_q11b_5,p_q11c_1,p_q11c_2,p_q11c_3,p_q11c_4,p_q11c_5,p_q11d_1,p_q11d_2,p_q11d_3,p_q11d_4,p_q11d_5,p_q11e_1,p_q11e_2,p_q11e_3,p_q11e_4,p_q11e_5,p_q12a_1,p_q12a_2,p_q12a_3,p_q12a_4,p_q12b_1,p_q12b_2,p_q12b_3,p_q12b_4,p_q12c_1,p_q12c_2,p_q12c_3,p_q12c_4,p_q12d_1,p_q12d_2,p_q12d_3,p_q12d_4,p_q12e_1,p_q12e_2,p_q12e_3,p_q12e_4,p_q12f_1,p_q12f_2,p_q12f_3,p_q12f_4,p_q13a_1,p_q13a_2,p_q13a_3,p_q13a_4,p_q13a_5,p_q13b_1,p_q13b_2,p_q13b_3,p_q13b_4,p_q13b_5,p_q13c_1,p_q13c_2,p_q13c_3,p_q13c_4,p_q13c_5,p_q13d_1,p_q13d_2,p_q13d_3,p_q13d_4,p_q13d_5,p_q14a_1,p_q14a_2,p_q14a_3,p_q14a_4,p_q14a_5,p_q14b_1,p_q14b_2,p_q14b_3,p_q14b_4,p_q14b_5,p_q14c_1,p_q14c_2,p_q14c_3,p_q14c_4,p_q14c_5,p_q14d_1,p_q14d_2,p_q14d_3,p_q14d_4,p_q14d_5,p_N_q1a_1,p_N_q1a_2,p_N_q1a_3,p_N_q1a_4,p_N_q1a_5,p_N_q1b_1,p_N_q1b_2,p_N_q1b_3,p_N_q1b_4,p_N_q1b_5,p_N_q1c_1,p_N_q1c_2,p_N_q1c_3,p_N_q1c_4,p_N_q1c_5,p_N_q1d_1,p_N_q1d_2,p_N_q1d_3,p_N_q1d_4,p_N_q1d_5,p_N_q1e_1,p_N_q1e_2,p_N_q1e_3,p_N_q1e_4,p_N_q1e_5,p_N_q1f_1,p_N_q1f_2,p_N_q1f_3,p_N_q1f_4,p_N_q1f_5,p_N_q1g_1,p_N_q1g_2,p_N_q1g_3,p_N_q1g_4,p_N_q1g_5,p_N_q2a_1,p_N_q2a_2,p_N_q2a_3,p_N_q2a_4,p_N_q2a_5,p_N_q2b_1,p_N_q2b_2,p_N_q2b_3,p_N_q2b_4,p_N_q2b_5,p_N_q3a_1,p_N_q3a_2,p_N_q3a_3,p_N_q3a_4,p_N_q3a_5,p_N_q3b_1,p_N_q3b_2,p_N_q3b_3,p_N_q3b_4,p_N_q3b_5,p_N_q3c_1,p_N_q3c_2,p_N_q3c_3,p_N_q3c_4,p_N_q3c_5,p_N_q3d_1,p_N_q3d_2,p_N_q3d_3,p_N_q3d_4,p_N_q3d_5,p_N_q3e_1,p_N_q3e_2,p_N_q3e_3,p_N_q3e_4,p_N_q3e_5,p_N_q4a_1,p_N_q4a_2,p_N_q4a_3,p_N_q4a_4,p_N_q4a_5,p_N_q4b_1,p_N_q4b_2,p_N_q4b_3,p_N_q4b_4,p_N_q4b_5,p_N_q4c_1,p_N_q4c_2,p_N_q4c_3,p_N_q4c_4,p_N_q4c_5,p_N_q5a,p_N_q5b,p_N_q5c,p_N_q5d,p_N_q5e,p_N_q5f,p_N_q5g,p_N_q5h,p_N_q5i,p_N_q5j,p_N_q5k,p_N_q5l,p_N_q5m,p_N_q5n,

### Combining the survey dataframes  

There is no need for us to separate the survey results for District 75 schools, so we will join `"d75_survey"` to the bottom of `"all_survey"` and create one final dataframe named `"survey"`.  

We will view the first and last rows to confirm that `"survey"` contains both Non-District 75 schools (`"dbn"` does not begin with `"75"`) and District 75 schools (`"dbn"` does begin with `"75"`).

In [10]:
# Join "d75_survey" to the bottom of "all_survey", then view the first
# and last rows of the new dataframe "survey".
survey = pd.concat([all_survey, d75_survey], axis=0, sort=False)
survey.iloc[[1,-1]]

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,N_s,N_t,N_p,nr_s,nr_t,nr_p,saf_p_11,com_p_11,eng_p_11,aca_p_11,saf_t_11,com_t_11,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11,p_q2h,p_q7a,p_q7b,p_q7c,p_q7d,p_q8a,p_q8b,p_q8c,p_q8d,p_q8e,p_q8f,p_q2b,p_q2d,p_q2e,p_q2f,p_q2g,p_q3a,p_q3b,p_q4b,p_q4c,p_q11c,p_q2a,p_q2c,p_q3c,p_q6a,p_q6b,p_q11d,p_q11e,p_q5,p_q4a,p_q4d,p_q4e,p_q11a,p_q11b,p_q11f,p_q1,p_q3d,p_q9,p_q10,p_q12aa,p_q12ab,p_q12ac,p_q12ad,p_q12ba,p_q12bb,p_q12bc,p_q12bd,p_q1_1,p_q1_2,p_q1_3,p_q1_4,p_q1_5,p_q1_6,p_q1_7,p_q1_8,p_q1_9,p_q1_10,p_q1_11,p_q1_12,p_q1_13,p_q1_14,p_q2a_1,p_q2a_2,p_q2a_3,p_q2a_4,p_q2a_5,p_q2b_1,p_q2b_2,p_q2b_3,p_q2b_4,p_q2b_5,p_q2c_1,p_q2c_2,p_q2c_3,p_q2c_4,p_q2c_5,p_q2d_1,p_q2d_2,p_q2d_3,p_q2d_4,p_q2d_5,p_q2e_1,p_q2e_2,p_q2e_3,p_q2e_4,p_q2e_5,p_q2f_1,p_q2f_2,p_q2f_3,p_q2f_4,p_q2f_5,p_q2g_1,p_q2g_2,p_q2g_3,p_q2g_4,p_q2g_5,p_q2h_1,p_q2h_2,p_q2h_3,p_q2h_4,p_q2h_5,p_q3a_1,p_q3a_2,p_q3a_3,p_q3a_4,p_q3a_5,p_q3b_1,p_q3b_2,p_q3b_3,p_q3b_4,p_q3b_5,p_q3c_1,p_q3c_2,p_q3c_3,p_q3c_4,p_q3c_5,p_q3d_1,p_q3d_2,p_q3d_3,p_q3d_4,p_q3d_5,p_q4a_1,p_q4a_2,p_q4a_3,p_q4a_4,p_q4a_5,p_q4b_1,p_q4b_2,p_q4b_3,p_q4b_4,p_q4b_5,p_q4c_1,p_q4c_2,p_q4c_3,p_q4c_4,p_q4c_5,p_q4d_1,p_q4d_2,p_q4d_3,p_q4d_4,p_q4d_5,p_q4e_1,p_q4e_2,p_q4e_3,p_q4e_4,p_q4e_5,p_q5a,p_q5b,p_q5c,p_q5d,p_q5e,p_q5f,p_q5g,p_q5h,p_q5i,p_q5j,p_q6a_1,p_q6a_2,p_q6a_3,p_q6a_4,p_q6a_5,p_q6b_1,p_q6b_2,p_q6b_3,p_q6b_4,p_q6b_5,p_q7a_1,p_q7a_2,p_q7a_3,p_q7a_4,p_q7a_5,p_q7b_1,p_q7b_2,p_q7b_3,p_q7b_4,p_q7b_5,p_q7c_1,p_q7c_2,p_q7c_3,p_q7c_4,p_q7c_5,p_q7d_1,p_q7d_2,p_q7d_3,p_q7d_4,p_q7d_5,p_q8a_1,p_q8a_2,p_q8a_3,p_q8a_4,p_q8a_5,p_q8b_1,p_q8b_2,p_q8b_3,p_q8b_4,p_q8b_5,p_q8c_1,p_q8c_2,p_q8c_3,p_q8c_4,p_q8c_5,p_q8d_1,p_q8d_2,p_q8d_3,p_q8d_4,p_q8d_5,p_q8e_1,p_q8e_2,p_q8e_3,p_q8e_4,p_q8e_5,p_q8f_1,p_q8f_2,p_q8f_3,p_q8f_4,p_q8f_5,p_q9_1,p_q9_2,p_q9_3,p_q9_4,p_q9_5,p_q9_6,p_q9_7,p_q9_8,p_q9_9,p_q9_10,p_q10a,p_q10b,p_q10c,p_q10d,p_q10e,p_q10f,p_q10g,p_q10h,p_q10i,p_q10j,p_q10k,p_q10l,p_q11a_1,p_q11a_2,p_q11a_3,p_q11a_4,p_q11b_1,p_q11b_2,p_q11b_3,p_q11b_4,p_q11c_1,p_q11c_2,p_q11c_3,p_q11c_4,p_q11d_1,p_q11d_2,p_q11d_3,p_q11d_4,p_q11e_1,p_q11e_2,p_q11e_3,p_q11e_4,p_q11f_1,p_q11f_2,p_q11f_3,p_q11f_4,p_q12aa_1,p_q12aa_2,p_q12aa_3,p_q12aa_4,p_q12aa_5,p_q12ab_1,p_q12ab_2,p_q12ab_3,p_q12ab_4,p_q12ab_5,p_q12ac_1,p_q12ac_2,p_q12ac_3,p_q12ac_4,p_q12ac_5,p_q12ad_1,p_q12ad_2,p_q12ad_3,p_q12ad_4,p_q12ad_5,p_q12ba_1,p_q12ba_2,p_q12ba_3,p_q12ba_4,p_q12ba_5,p_q12bb_1,p_q12bb_2,p_q12bb_3,p_q12bb_4,p_q12bb_5,p_q12bc_1,p_q12bc_2,p_q12bc_3,p_q12bc_4,p_q12bc_5,p_q12bd_1,p_q12bd_2,p_q12bd_3,p_q12bd_4,p_q12bd_5,p_N_q1_1,p_N_q1_2,p_N_q1_3,p_N_q1_4,p_N_q1_5,p_N_q1_6,p_N_q1_7,p_N_q1_8,p_N_q1_9,p_N_q1_10,p_N_q1_11,p_N_q1_12,p_N_q1_13,p_N_q1_14,p_N_q2a_1,p_N_q2a_2,p_N_q2a_3,p_N_q2a_4,p_N_q2a_5,p_N_q2b_1,p_N_q2b_2,p_N_q2b_3,p_N_q2b_4,p_N_q2b_5,p_N_q2c_1,p_N_q2c_2,p_N_q2c_3,p_N_q2c_4,p_N_q2c_5,p_N_q2d_1,p_N_q2d_2,p_N_q2d_3,p_N_q2d_4,p_N_q2d_5,p_N_q2e_1,p_N_q2e_2,p_N_q2e_3,p_N_q2e_4,p_N_q2e_5,p_N_q2f_1,p_N_q2f_2,p_N_q2f_3,p_N_q2f_4,p_N_q2f_5,p_N_q2g_1,p_N_q2g_2,p_N_q2g_3,p_N_q2g_4,p_N_q2g_5,p_N_q2h_1,p_N_q2h_2,p_N_q2h_3,p_N_q2h_4,p_N_q2h_5,p_N_q3a_1,p_N_q3a_2,p_N_q3a_3,p_N_q3a_4,p_N_q3a_5,p_N_q3b_1,p_N_q3b_2,p_N_q3b_3,p_N_q3b_4,p_N_q3b_5,p_N_q3c_1,p_N_q3c_2,p_N_q3c_3,p_N_q3c_4,p_N_q3c_5,p_N_q3d_1,p_N_q3d_2,p_N_q3d_3,p_N_q3d_4,p_N_q3d_5,p_N_q4a_1,p_N_q4a_2,p_N_q4a_3,p_N_q4a_4,p_N_q4a_5,p_N_q4b_1,p_N_q4b_2,p_N_q4b_3,p_N_q4b_4,p_N_q4b_5,p_N_q4c_1,p_N_q4c_2,p_N_q4c_3,p_N_q4c_4,p_N_q4c_5,p_N_q4d_1,p_N_q4d_2,p_N_q4d_3,p_N_q4d_4,p_N_q4d_5,p_N_q4e_1,p_N_q4e_2,p_N_q4e_3,p_N_q4e_4,p_N_q4e_5,p_N_q5a,p_N_q5b,p_N_q5c,p_N_q5d,p_N_q5e,p_N_q5f,p_N_q5g,p_N_q5h,p_N_q5i,p_N_q5j,p_N_q6a_1,p_N_q6a_2,p_N_q6a_3,p_N_q6a_4,p_N_q6a_5,p_N_q6b_1,p_N_q6b_2,p_N_q6b_3,p_N_q6b_4,p_N_q6b_5,p_N_q7a_1,p_N_q7a_2,p_N_q7a_3,p_N_q7a_4,p_N_q7a_5,p_N_q7b_1,p_N_q7b_2,p_N_q7b_3,p_N_q7b_4,p_N_q7b_5,p_N_q7c_1,p_N_q7c_2,p_N_q7c_3,p_N_q7c_4,p_N_q7c_5,p_N_q7d_1,p_N_q7d_2,p_N_q7d_3,p_N_q7d_

# Cleaning the `"survey"` dataframe  

In `"survey"`, we will copy the values of `"dbn"` into a new `"DBN"` column (all capital letters) so it is consistent with the other dataframes.  

There are 2,773 columns in `"survey"` but we will only need a select few for our future analysis.  Based on reviewing `"Survey_Data_Dictionary.xls"` and hypothesizing which data could possibly influence SAT results, we will only keep the following columns:  

* `"DBN"`: School identification code (district, borough, school number)  


* `"rr_s"`: Student response rate  
* `"rr_t"`: Teacher response rate  
* `"rr_p"`: Parent response rate  


* `"N_s"`: Number of student respondents  
* `"N_t"`: Number of teacher respondents  
* `"N_p"`: Number of parent respondents  


* `"saf_s_11"`: Safety and respect score based on student responses  
* `"saf_t_11"`: Safety and respect score based on teacher responses
* `"saf_p_11"`: Safety and respect score based on parent responses  
* `"saf_tot_11"`: Safety and respect total score  


* `"eng_s_11"`: Engagement score based on student responses  
* `"eng_t_11"`: Engagement score based on teacher responses  
* `"eng_p_11"`: Engagement score based on parent responses
* `"eng_tot_11"`: Engagement total score  


* `"aca_s_11"`: Academic expectations score based on student responses  
* `"aca_t_11"`: Academic expectations score based on teacher responses  
* `"aca_p_11"`: Academic expectations score based on parent responses
* `"aca_tot_11"`: Academic expectations total score


* `"com_s_11"`: Communication score based on student responses  
* `"com_t_11"`: Communication score based on teacher responses
* `"com_p_11"`: Communication score based on parent responses
* `"com_tot_11"`: Communication total score  

After filtering `"survey"` to the selected columns above, we will add it to our `"data"` dictionary along with the other dataframes.  We will then view the first 5 rows of `"survey"`.

In [11]:
# Create the "DBN" column, filter "survey" to our selected columns, add
# it to our "data" dictionary and view the first 5 rows.
survey_cols = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p",
               "saf_s_11", "saf_t_11", "saf_p_11", "saf_tot_11",
               "eng_s_11", "eng_t_11", "eng_p_11", "eng_tot_11",
               "aca_s_11", "aca_t_11", "aca_p_11", "aca_tot_11",
               "com_s_11", "com_t_11", "com_p_11", "com_tot_11"]
survey["DBN"] = survey["dbn"]
survey = survey.loc[:, survey_cols]
data["survey"] = survey
data["survey"].head()

,DBN,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_s_11,saf_t_11,saf_p_11,saf_tot_11,eng_s_11,eng_t_11,eng_p_11,eng_tot_11,aca_s_11,aca_t_11,aca_p_11,aca_tot_11,com_s_11,com_t_11,com_p_11,com_tot_11
0,01M015,NaN,88,60,NaN,22.0,90.0,NaN,7.5,8.5,8.0,NaN,7.6,7.5,7.5,NaN,7.9,7.8,7.9,NaN,7.8,7.6,7.7
1,01M019,NaN,100,60,NaN,34.0,161.0,NaN,8.6,8.4,8.5,NaN,8.9,7.6,8.2,NaN,9.1,7.8,8.4,NaN,8.5,7.6,8.1
2,01M020,NaN,88,73,NaN,42.0,367.0,NaN,7.6,8.9,8.2,NaN,6.8,8.3,7.5,NaN,7.5,8.6,8.0,NaN,6.3,8.3,7.3
3,01M034,89.0,73,50,145.0,29.0,151.0,6.2,7.0,8.8,7.3,6.5,6.8,8.0,7.1,7.4,7.8,8.5,7.9,5.9,6.2,8.2,6.7
4,01M063,NaN,100,60,NaN,23.0,90.0,NaN,8.4,8.7,8.5,NaN,7.8,8.1,7.9,NaN,8.1,7.9,8.0,NaN,7.3,7.9,7.6


# Inserting `"DBN"` columns  

### Inserting the `"DBN"` column into `"hs_directory"`

In `"hs_directory"`, we will copy the values of the `"dbn"` column into a new `"DBN"` column (all capital letters) so it is consistent with the other dataframes.  We will then view the first 5 rows to confirm it was created properly.  The new `"DBN"` column will be at the right most end of `"hs_directory"`.

In [12]:
# Create the "DBN" column and view the first 5 rows.
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]
data["hs_directory"].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1,DBN
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,"B41, B43, B44-SBS, B45, B48, B49, B69","2, 3, 4, 5, F, S to Botanic Garden ; B, Q to P...",883 Classon Avenue,Brooklyn,NY,11225,Bkmusicntheatre.com,399.0,Prospect Heights Educational Campus,NaN,Brooklyn School for Music & Theatre (BSMT) use...,We offer highly competitive positions in our D...,Spanish,"English Language and Composition, United State...",NaN,NaN,"Variety of clubs: Chess, The Step Team, Fashio...","Baseball, Basketball & JV Basketball, Cross Co...","Basketball, Cross Country, Indoor Track, Outdo...",NaN,NaN,F.Y.R.EZONE (Finding Your Rhythm thru Educatio...,NaN,NaN,"In 2002, Roundabout Theatre was selected by Ne...",One To World‘s Global Classroom connects New Y...,NaN,NaN,NaN,NaN,NaN,8:10 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Brooklyn students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",17K548
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,"Bx13, Bx15, Bx17, Bx21, Bx35, Bx4, Bx41, Bx4A,...","2, 5 to Intervale Ave",1110 Boston Road,Bronx,NY,10456,www.hsvd.org,378.0,Morris Educational Campus,NaN,The High School for Violin and Dance (HSVD) is...,Freshmen take both violin and dance; College N...,Spanish,NaN,NaN,NaN,Advancement via Individual Determination (AVID...,"Baseball, Basketball & JV Basketball, Volleyball","Basketball, Softball, Volleyball",NaN,Morris Educational Campus Basketball and Volle...,McGraw Hill - Big Brother Big Sister,NaN,"Hostos Community College, Monroe College, Teac...",Bronx Arts Ensemble,buildOn,Print International,NaN,Bronx Cares,Our students are required to take four years o...,"Student Summer Orientation, Summer Internship ...",8:00 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Bronx students or residents who at...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",09X543
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,"Bx1, Bx11, Bx13, Bx18, Bx2, Bx3, Bx32, Bx35, Bx36","4 to Mt Eden Ave ; B, D to 170th St",1501 Jerome Avenue,Bronx,NY,10452,http://schools.nyc.gov/schoolportals/09/X327,543.0,DOE New Settlement Community Campus,NaN,At the Comprehensive Model School Project (CMS...,"After-school and Saturday Tutoring, Advisory, ...",Spanish,"Biology, Chemistry, United States History",NaN,NaN,"Choir, Gaming, Girls Club, Newspaper, Spanish,...",NaN,NaN,NaN,"As we expand, we plan to offer PSAL sports.",New Settlement Community Center,Montefiore Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Dress Code Required: white or baby blue button...,NaN,8:00 AM,4:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to continuing 8th graders,Then to Bronx students or residents who attend...,Then to New York City residents who attend an ...,

### Inserting the `"DBN"` column into `"class_size"`  

As we discovered previously, `"class_size"` doesn't have a `"DBN"` column but it can be created by joining the `"CSD"` and `"SCHOOL CODE"` columns.  If `"CSD"` is only a single digit, we need to add a leading `"0"` to make the `"DBN"` consistent with the other dataframes.  

We will create a function named `pad_csd` which takes in an integer value, converts it to a string, adds a leading `"0"` (if it is only a single digit) and returns the string.  We will then apply this function to the `"CSD"` column to create a new column named `"padded_csd"`, which will contain all of the `"CSD"` values as 2-digit integer strings.  Finally, we will combine the `"padded_csd"` and `"SCHOOL CODE"` columns to create the `"DBN"` column.  

We will then view the first 5 rows to confirm both columns were created properly. The new `"padded_csd"` and `"DBN"` columns will be at the right most end of `"class_size"`.

In [13]:
def pad_csd(int):
    """Convert an integer to a 2-digit string and return the string."""
    string = str(int)
    if len(string) == 2:
        return string
    else:
        return string.zfill(2)

# Create "padded_csd" and "DBN" columns and view the first 5 rows.
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = (data["class_size"]["padded_csd"]
                             + data["class_size"]["SCHOOL CODE"])
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015


# Cleaning the `"sat_results"` dataframe

With the exception of `"DBN"` and `"SCHOOL NAME"`, every column in `"sat_results"` should have a numeric data type (`float` or `int`). We will view the data types of each column to confirm.

In [14]:
# View the column data types of "sat_results".
data["sat_results"].dtypes

DBN                                object
SCHOOL NAME                        object
Num of SAT Test Takers             object
SAT Critical Reading Avg. Score    object
SAT Math Avg. Score                object
SAT Writing Avg. Score             object
dtype: object

We will convert the last 4 columns to a numeric data type.

Since we are only interested in the total SAT score for each school, we will add the separate SAT score columns together (critical reading, math and writing) to create a new column of total SAT scores named `"sat_score"`.  Finally, we will view the first 5 rows to confirm the new column was created properly.

In [15]:
# Convert the last 4 columns to a numeric data type, create the
# "sat_score" column and view the first 5 rows.
for col in data["sat_results"].columns[2:]:
    data["sat_results"][col] = pd.to_numeric(data["sat_results"][col],
                                             errors="coerce")
data["sat_results"]["sat_score"] = (
    data["sat_results"]["SAT Critical Reading Avg. Score"]
    + data["sat_results"]["SAT Math Avg. Score"]
    + data["sat_results"]["SAT Writing Avg. Score"]
)
data["sat_results"].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,390.0,433.0,384.0,1207.0


# Parsing geographic coordinates in `"hs_directory"`  

As discussed earlier, the `"Location 1"` column of `"hs_directory"` contains the geographic coordinates (latitude, longitude) of each school.  We are going to extract these latitudes (lat) and longitudes (lon) so that we will be able to plot the SAT results of each school on a map of NYC.

### Extracting and converting the latitude  

In the `"Location 1"` column, the geographic coordinates for each school are contained within parentheses `()`.  The latitude is the first coordinate.  

We will import the `re` (regular expression) module and create a function named `get_lat`.  This function will take in a string, find all of the characters that are contained within parentheses (the coordinates), select the first coordinate (the latitude), drop the parentheses and return the cleaned latitude value.  

We will then apply this function to the `"Location 1"` column to extract all of the cleaned latitude values and populate a new column named `"lat"`.  We will convert this column from a string data type to a numeric data type so that arithmetic operations can be performed on it.  We will then view the first 5 rows of the `"Location 1"` and `"lat"` columns to confirm the new column was created properly.

In [16]:
import re

def get_lat(string):
    """Find, clean and return the lat coordinates from a string."""
    coord = re.findall("\(.+\)", string)[0]
    lat = coord.split(", ")[0]
    lat_clean = lat.replace("(", "").replace(")", "")    
    return lat_clean

# Create the "lat" column, convert to a numeric data type and view the
# first 5 rows of the relevant columns.
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(get_lat)
data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"],
                                            errors="coerce")
data["hs_directory"][["Location 1", "lat"]].head()

,Location 1,lat
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679
4,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810


### Extracting and converting the longitude  

All of the steps here are analogous to the ["Extracting and converting the latitude"](#Extracting-and-converting-the-latitude) steps above, except we are extracting the 2nd coordinate, the longitude, and creating the new column `"lon"`.  

We will view the first 5 rows of the `"Location 1"`, `"lat"` and `"lon"` columns to confirm everything looks correct.

In [17]:
def get_lon(string):
    """Find, clean and return the lon coordinates from a string."""
    coord = re.findall("\(.+\)", string)[0]
    lon = coord.split(", ")[1]
    lon_clean = lon.replace("(", "").replace(")", "")    
    return lon_clean

# Create the "lon" column, convert to a numeric data type and view the 
# first 5 rows of the relevant columns.
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(get_lon)
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"],
                                            errors="coerce")
data["hs_directory"][["Location 1", "lat", "lon"]].head()

,Location 1,lat,lon
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299,-73.961648
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603,-73.904475
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414,-73.916162
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679,-74.000807
4,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810,-73.806500


# Cleaning the `"hs_directory"` dataframe 

We are only interested in the `"DBN"`, `"lat"` and `"lon"` columns of `"hs_directory"` so we will filter the dataframe to only include these columns.

In [18]:
# Filter "hs_directory" to our selected columns and view the first 5
# rows.
data["hs_directory"] = data["hs_directory"][["DBN", "lat", "lon"]]
data["hs_directory"].head()

,DBN,lat,lon
0,17K548,40.670299,-73.961648
1,09X543,40.827603,-73.904475
2,09X327,40.842414,-73.916162
3,02M280,40.710679,-74.000807
4,28Q680,40.718810,-73.806500


# Cleaning the `"class_size"` dataframe  

As we discussed earlier when exploring `"class_size"`, each row of the dataframe represents a specific `"GRADE"` and `"PROGRAM TYPE"` for that school.  We will have to filter for the rows that contain the `"GRADE"` and `"PROGRAM TYPE"` values that are most suitable for our analysis.  

For ease and convenience we are first going to assign the `"class_size"` dataframe in our `"data"` dictionary to a variable named `"class_size"`.  We will then view the unique values of the `"GRADE "` column.

In [19]:
# Assign the "class_size" dataframe to a variable and view the unique
# values in the "GRADE " column.
class_size = data["class_size"]
list(class_size["GRADE "].unique())

['0K',
 '01',
 '02',
 '03',
 '04',
 '05',
 '0K-09',
 nan,
 '06',
 '07',
 '08',
 'MS Core',
 '09-12',
 '09']

Similarly, we will view the unique values of the `"PROGRAM TYPE"` column.

In [20]:
# View the unique values in the "PROGRAM TYPE" column of "class_size".
list(class_size["PROGRAM TYPE"].unique())

['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T']

Since we are only exploring high school data, we will filter `"class_size"` to only include rows where the `"GRADE"` column is `"09-12"`.  

Referencing `"Class_Size_Data_Dictionary.xlsx"` in the `"references"` folder of this project, the `"PROGRAM TYPE"` values of `"CTT"` (collaborative team teaching), `"SPEC ED"` (self-contained special education) and `"G&T"` (gifted and talented) are all specialty programs.  So we will filter the dataframe to only include the `"GEN ED"` (general education) program type.  

We will then view the first 5 rows of `"class_size"` to confirm it's only including the rows we selected.

In [21]:
# Filter "class_size" to rows with our selected "GRADE" and
# "PROGRAM TYPE" values and view the first 5 rows.
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]
class_size.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01,01M292


### Creating a unique `"DBN"` for each row in `"class_size"`  

Each row in `"class_size"` now represents a specific `"CORE COURSE"` for that school (i.e. English 9, English 10, Integrated Algebra etc...).  This means there are still `"DBN"` values repeated across multiple rows.  We need to aggregate the values in `"class_size"` so there is a unique `"DBN"` in each row.  

We will accomplish this by grouping `"class_size"` by each unique `"DBN"` value and aggregating the values by calculating the mean of each column within each `"DBN"` group.  We will then reset the index of the new dataframe.  

The `"CSD"` column will not be needed for our analysis and the `"SCHOOLWIDE PUPIL-TEACHER RATIO"` column contains all missing values, so we will drop both of those columns from the dataframe.  

We will assign the new `"class_size"` dataframe, with unique `"DBN"` values in each row, back to our `"data"` dictionary and view the first 5 rows.

In [22]:
import numpy as np

# Group "class_size" by "DBN", reset the index, drop the selected
# columns, assign the dataframe back to our "data" dictionary and 
# view the first 5 rows.
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
class_size.drop(columns=["CSD", "SCHOOLWIDE PUPIL-TEACHER RATIO"],
                inplace=True)
data["class_size"] = class_size
data["class_size"].head()

,DBN,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS
0,01M292,88.0000,4.000000,22.564286,18.50,26.571429
1,01M332,46.0000,2.000000,22.000000,21.00,23.500000
2,01M378,33.0000,1.000000,33.000000,33.00,33.000000
3,01M448,105.6875,4.750000,22.231250,18.25,27.062500
4,01M450,57.6000,2.733333,21.200000,19.40,22.866667


# Cleaning the `"demographics"` dataframe  

As we discussed earlier when exploring `"demographics"`, each row represents a specific school year for that school. We will have to filter for the rows that contain the school years that are most suitable for our analysis.  

First we will examine the unique values in the `"schoolyear"` column of `"demographics"`.

In [23]:
# View the unique values in the "schoolyear" column of "demographics".
list(data["demographics"]["schoolyear"].unique())

[20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012]

We want the most recent data available, so we will filter the dataframe to only include the `"schoolyear"` value of `"20112012"`.  This will also ensure each row has a unique `"DBN"` value.  

We also don't need `"Name"`, `"schoolyear"` or any of the columns describing the grade level, so we will drop them from the dataframe.

We will then view the first 5 rows to confirm the dataframe has been condensed properly.

In [24]:
# Filter "demographics" to the "schoolyear" value of "20112012", drop
# the selected columns and view the first 5 rows.
data["demographics"] = (
    data["demographics"][data["demographics"]["schoolyear"] == 20112012]
)
drop_cols = ["Name", "schoolyear", "prek", "k", "grade1", "grade2", "grade3",
             "grade4", "grade5", "grade6", "grade7", "grade8", "grade9", 
             "grade10", "grade11", "grade12"]
data["demographics"].drop(columns=drop_cols, inplace=True)
data["demographics"].head()

,DBN,fl_percent,frl_percent,total_enrollment,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,NaN,89.4,189,20.0,10.6,40.0,21.2,23,7,12,6.3,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,NaN,61.5,328,33.0,10.1,59.0,18.0,16,16,51,15.5,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,NaN,92.5,626,128.0,20.4,97.0,15.5,49,31,190,30.4,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,NaN,99.7,401,34.0,8.5,106.0,26.4,59,16,22,5.5,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,NaN,78.9,176,6.0,3.4,45.0,25.6,34,4,9,5.1,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


### Correcting column data types  

With the exception of `"DBN"`, every column in `"demographics"` should have a numeric data type (`float` or `int`).  We will view the data types of each column to confirm.

In [25]:
# View the column data types of "demographics".
data["demographics"].dtypes

DBN                   object
fl_percent            object
frl_percent          float64
total_enrollment       int64
ell_num              float64
ell_percent          float64
sped_num             float64
sped_percent         float64
ctt_num               object
selfcontained_num     object
asian_num              int64
asian_per            float64
black_num              int64
black_per            float64
hispanic_num           int64
hispanic_per         float64
white_num              int64
white_per            float64
male_num             float64
male_per             float64
female_num           float64
female_per           float64
dtype: object

The `"fl_percent"`, `"ctt_num"` and `"selfcontained_num"` columns have `object` data types. Upon further inspection, `"fl_percent"` contains all missing values, so we will drop that column.  We will then convert the other 2 columns to numeric data types.  

Finally, we will view the column data types again to confirm everything looks correct.

In [26]:
# Drop the "fl_percent" column, convert "ctt_num" and
# "selfcontained_num" to numeric data types and view the column data
# types.
data["demographics"].drop(columns="fl_percent", inplace=True)
for col in ["ctt_num", "selfcontained_num"]:
    data["demographics"][col] = pd.to_numeric(data["demographics"][col],
                                              errors="coerce")
data["demographics"].dtypes

DBN                   object
frl_percent          float64
total_enrollment       int64
ell_num              float64
ell_percent          float64
sped_num             float64
sped_percent         float64
ctt_num              float64
selfcontained_num    float64
asian_num              int64
asian_per            float64
black_num              int64
black_per            float64
hispanic_num           int64
hispanic_per         float64
white_num              int64
white_per            float64
male_num             float64
male_per             float64
female_num           float64
female_per           float64
dtype: object

# Cleaning the `"graduation"` dataframe  

As we discussed earlier when exploring `"graduation"`, each row represents a specific `"Demographic"` and `"Cohort"` for that school. We will have to filter for the rows that contain the `"Demographic"` and `"Cohort"` values that are most suitable for our analysis.

First we will examine the unique values in the `"Demographic"` column of `"graduation"`.

In [27]:
# View the unique values in the "Demographic" column of "graduation".
list(data["graduation"]["Demographic"].unique())

['Total Cohort',
 'Male',
 'English Language Learners',
 'English Proficient Students',
 'Special Education Students',
 'General Education Students',
 'Asian',
 'Black',
 'Hispanic',
 'White',
 'Female']

Similarly, we will view the unique values in the `"Cohort"` column.

In [28]:
# View the unique values in the "Cohort" column of "graduation".
list(data["graduation"]["Cohort"].unique())

['2003', '2004', '2005', '2006', '2006 Aug', '2001', '2002']

We already have demographic specific data in the `"demographics"` dataframe, so we will filter `"graduation"` to include only the `"Demographic"` value of `"Total Cohort"`.  We want the most recent full-year data available, so we will filter the dataframe to include only the `"Cohort"` value of `"2006"`.  Both of these filters will also ensure that each row has a unique `"DBN"` value.  

We will then drop the `"Demographic"`, `"School Name"` and `"Cohort"` columns as they will no longer be needed for our future analysis.

We will view the first 5 rows to confirm the dataframe has been condensed properly.

In [29]:
# Filter "graduation" to rows with our selected "Demographic" and
# "Cohort" values, drop the selected columns and view the first 5 rows.
data["graduation"] = (
    data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]
)
data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
drop_cols = ["Demographic", "School Name", "Cohort"]
data["graduation"].drop(columns=drop_cols, inplace=True)
data["graduation"].head()

,DBN,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,01M292,78,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
10,01M448,124,53,42.7%,42,33.9%,79.2%,8,6.5%,15.1%,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
17,01M450,90,70,77.8%,67,74.400000000000006%,95.7%,0,0%,0%,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
24,01M509,84,47,56%,40,47.6%,85.1%,17,20.2%,36.200000000000003%,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%
31,01M515,193,105,54.4%,91,47.2%,86.7%,69,35.799999999999997%,65.7%,22,11.4%,21%,14,7.3%,13.3%,53,27.5%,35,18.100000000000001%


### Correcting column data types  

With the exception of `"DBN"`, every column in `"graduation"` should have a numeric data type (`float` or `int`). We will view the data types of each column to confirm.

In [30]:
# View the column data types of "graduation".
data["graduation"].dtypes

DBN                                   object
Total Cohort                           int64
Total Grads - n                       object
Total Grads - % of cohort             object
Total Regents - n                     object
Total Regents - % of cohort           object
Total Regents - % of grads            object
Advanced Regents - n                  object
Advanced Regents - % of cohort        object
Advanced Regents - % of grads         object
Regents w/o Advanced - n              object
Regents w/o Advanced - % of cohort    object
Regents w/o Advanced - % of grads     object
Local - n                             object
Local - % of cohort                   object
Local - % of grads                    object
Still Enrolled - n                    object
Still Enrolled - % of cohort          object
Dropped Out - n                       object
Dropped Out - % of cohort             object
dtype: object

The third to the last columns need to be converted to numeric data types.  We will remove all of the `%` symbols from those columns and then convert them from string to numeric data types.  We will then view the first 5 rows to confirm everything looks right.

In [31]:
# Remove all "%" symbols from the third to the last columns, convert
# each column to a numeric data type and view the first 5 rows.
for col in data["graduation"].columns[2:]: 
    data["graduation"][col] = (
        data["graduation"][col].apply(lambda x: x.replace("%",""))
    )
    data["graduation"][col] = pd.to_numeric(data["graduation"][col],
                                            errors="coerce")
data["graduation"].head()

,DBN,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,01M292,78,43.0,55.1,36.0,46.2,83.7,0.0,0.0,0.0,36.0,46.2,83.7,7.0,9.0,16.3,16.0,20.5,11.0,14.1
10,01M448,124,53.0,42.7,42.0,33.9,79.2,8.0,6.5,15.1,34.0,27.4,64.2,11.0,8.9,20.8,46.0,37.1,20.0,16.1
17,01M450,90,70.0,77.8,67.0,74.4,95.7,0.0,0.0,0.0,67.0,74.4,95.7,3.0,3.3,4.3,15.0,16.7,5.0,5.6
24,01M509,84,47.0,56.0,40.0,47.6,85.1,17.0,20.2,36.2,23.0,27.4,48.9,7.0,8.3,14.9,25.0,29.8,5.0,6.0
31,01M515,193,105.0,54.4,91.0,47.2,86.7,69.0,35.8,65.7,22.0,11.4,21.0,14.0,7.3,13.3,53.0,27.5,35.0,18.1


# Cleaning the `"ap_results"` dataframe  

With the exception of `"DBN"`, every column in `"ap_results"` should have a numeric data type (`float` or `int`). We will view the data types of each column to confirm.

In [32]:
# View the column data types of "ap_results".
data["ap_results"].dtypes

DBN                            object
SCHOOL NAME                    object
Num of AP Test Takers          object
Num of AP Total Exams Taken    object
Num of AP Exams Passed         object
dtype: object

We will drop the `"SCHOOL NAME"` column from `"ap_results"` as it will not be needed for our future analysis.

We will convert the last 3 columns to a numeric data type and then view the data types for all of the columns to confirm everything looks right.

In [33]:
# Drop the "SCHOOL NAME" column, convert the last 3 columns to a numeric
# data type and view the data types of each column in "ap_results".
data["ap_results"].drop(columns="SCHOOL NAME", inplace=True)
for col in data["ap_results"].columns[1:]:
    data["ap_results"][col] = pd.to_numeric(data["ap_results"][col],
                                            errors="coerce")
data["ap_results"].dtypes

DBN                             object
Num of AP Test Takers          float64
Num of AP Total Exams Taken    float64
Num of AP Exams Passed         float64
dtype: object

# Confirming unique `"DBN"` values for each dataframe  

Each dataframe should have a unique `"DBN"` value in each row.  To check this, we will print the name of each dataframe along with a list of any repeated `"DBN"` values and their counts.

In [34]:
# List any repeated "DBN" values and their counts for each dataframe.
for dataframe in data.keys():
    thresh = data[dataframe]["DBN"].value_counts() > 1
    multiple_dbn = list(data[dataframe]["DBN"].value_counts()[thresh].items())
    print(dataframe, multiple_dbn)

ap_results []
class_size []
demographics []
graduation []
hs_directory []
sat_results []
survey []


The empty lists confirm that each dataframe has a unique `"DBN"` value in each row.

# Joining all of the dataframes  

All of the dataframes are now ready to be joined into a single dataframe using the `"DBN"` columns.  To determine which join methods to use, we will first look at the number of rows in each dataframe.

In [35]:
# View the number of rows for each dataframe in the "data" dictionary.
for dataframe in data.keys():
    print(dataframe + ": " + str(data[dataframe].shape[0]))

ap_results: 478
class_size: 583
demographics: 1509
graduation: 405
hs_directory: 435
sat_results: 478
survey: 1702


### Performing the left joins  

The `"sat_results"` dataframe contains our target variable of SAT scores for each school.  The ultimate purpose for this final cleaned dataframe is to explore any correlations between SAT scores and the other datasets.  Another potential application is predicting SAT scores by training machine learning algorithms on the other data.  Therefore, we want to preserve as much of the `"sat_results"` data as possible.  

Since `"graduation"` has considerably less rows than `"sat_results"`, we will left join it onto `"sat_results"` to create a new dataframe named `"combined"`.  The left join will preserve all of the rows from `"sat_results"` and merge in any rows from `"graduation"` that has a matching `"DBN"` value with `"sat_results"`.  

Any rows with `"DBN"` values that exist in `"sat_results"` but not in `"graduation"` will be filled with `NaN` (not a number) missing values in the respective columns of `"graduation"`.  Since `"graduation"` only contains quantitative data, we can use quantitative methods to fill in those missing values (i.e. K-Nearest Neighbors).

We will view the first 5 rows of the new `"combined"` dataframe.

In [36]:
# Left join "graduation" onto "sat_results" to create the "combined"
# dataframe and view the first 5 rows.
combined = data["sat_results"]
combined = combined.merge(data["graduation"], how="left", on="DBN")
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,355.0,404.0,363.0,1122.0,78.0,43.0,55.1,36.0,46.2,83.7,0.0,0.0,0.0,36.0,46.2,83.7,7.0,9.0,16.3,16.0,20.5,11.0,14.1
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,383.0,423.0,366.0,1172.0,124.0,53.0,42.7,42.0,33.9,79.2,8.0,6.5,15.1,34.0,27.4,64.2,11.0,8.9,20.8,46.0,37.1,20.0,16.1
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,377.0,402.0,370.0,1149.0,90.0,70.0,77.8,67.0,74.4,95.7,0.0,0.0,0.0,67.0,74.4,95.7,3.0,3.3,4.3,15.0,16.7,5.0,5.6
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,414.0,401.0,359.0,1174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,390.0,433.0,384.0,1207.0,84.0,47.0,56.0,40.0,47.6,85.1,17.0,20.2,36.2,23.0,27.4,48.9,7.0,8.3,14.9,25.0,29.8,5.0,6.0


### Performing the inner joins  

With the exception of `"hs_directory"`, the remaining dataframes have at least as many rows as `"sat_results"`, so we will inner join them onto `"combined"`.  Using inner joins means only rows with matching `"DBN"` values between `"sat_results"` and the remaining dataframes will be included.  This prevents adding rows with missing values that we will have to deal with, while the equivalent or larger sizes of the remaining dataframes protects against losing too much `"sat_results"` data.

Even though `"hs_directory"` has less rows than `"sat_results"` we are still performing an inner join, which will result in some lost `"sat_results"` data.  This is because `"hs_directory"` contains only descriptive data (i.e. geographic coordinates) and if we use a left join we would not have a reliable method to fill in the missing values (i.e. K-Nearest Neighbors).  

After performing the inner joins, we will view the first 5 rows of `"combined"`.

In [37]:
# Inner join the remaining dataframes to "combined" and view the first
# 5 rows.
join_dataframes = ["ap_results", "class_size", "demographics", "survey",
                   "hs_directory"]
for dataframe in join_dataframes:
    combined = combined.merge(data[dataframe], how="inner", on="DBN")
combined.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort,Num of AP Test Takers,Num of AP Total Exams Taken,Num of AP Exams Passed,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,frl_percent,total_enrollment,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_s_11,saf_t_11,saf_p_11,saf_tot_11,eng_s_11,eng_t_11,eng_p_11,eng_tot_11,aca_s_11,aca_t_11,aca_p_11,aca_tot_11,com_s_11,com_t_11,com_p_11,com_tot_11,lat,lon
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,355.0,404.0,363.0,1122.0,78.0,43.0,55.1,36.0,46.2,83.7,0.0,0.0,0.0,36.0,46.2,83.7,7.0,9.0,16.3,16.0,20.5,11.0,14.1,NaN,NaN,NaN,88.000000,4.000000,22.564286,18.500000,26.571429,88.6,422,94.0,22.3,105.0,24.9,34.0,35.0,59,14.0,123,29.1,227,53.8,7,1.7,259.0,61.4,163.0,38.6,89.0,70,39,379.0,26.0,151.0,6.0,6.3,7.8,6.7,6.1,6.1,7.4,6.6,6.7,6.5,7.6,7.0,5.6,5.3,7.7,6.2,40.713764,-73.985260
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,383.0,423.0,366.0,1172.0,124.0,53.0,42.7,42.0,33.9,79.2,8.0,6.5,15.1,34.0,27.4,64.2,11.0,8.9,20.8,46.0,37.1,20.0,16.1,37.0,53.0,21.0,105.687500,4.750000,22.231250,18.250000,27.062500,71.8,394,83.0,21.1,86.0,21.8,55.0,10.0,115,29.2,89,22.6,181,45.9,9,2.3,226.0,57.4,168.0,42.6,84.0,95,10,385.0,37.0,46.0,6.0,6.6,7.9,6.8,6.3,6.6,7.2,6.7,7.0,7.3,7.3,7.2,5.7,5.8,7.4,6.3,40.712332,-73.984797
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,377.0,402.0,370.0,1149.0,90.0,70.0,77.8,67.0,74.4,95.7,0.0,0.0,0.0,67.0,74.4,95.7,3.0,3.3,4.3,15.0,16.7,5.0,5.6,12.0,12.0,NaN,57.600000,2.733333,21.200000,19.400000,22.866667,71.8,598,30.0,5.0,158.0,26.4,91.0,19.0,58,9.7,143,23.9,331,55.4,62,10.4,327.0,54.7,271.0,45.3,0.0,98,28,NaN,42.0,150.0,NaN,7.3,8.7,7.9,NaN,8.0,8.1,7.9,NaN,8.8,8.4,8.4,NaN,8.0,8.2,7.9,40.729783,-73.983041
3,01M509,MARTA VALLE HIGH SCHOOL,44.0,390.0,433.0,384.0,1207.0,84.0,47.0,56.0,40.0,47.6,85.1,17.0,20.2,36.2,23.0,27.4,48.9,7.0,8.3,14.9,25.0,29.8,5.0,6.0,14.0,15.0,NaN,69.642857,3.000000,23.571429,20.000000,27.357143,80.7,367,41.0,11.2,95.0,25.9,28.0,36.0,34,9.3,116,31.6,209,56.9,6,1.6,170.0,46.3,197.0,53.7,90.0,100,21,306.0,29.0,69.0,6.4,6.4,7.7,6.9,6.4,6.1,7.2,6.6,7.0,6.8,7.3,7.0,5.9,5.3,7.4,6.2,40.720569,-73.985673
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159.0,522.0,574.0,525.0,1621.0,46.0,46.0,100.0,46.0,100.0,100.0,31.0,67.4,67.4,15.0,32.6,32.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0,587.0,323.0,156.368421,6.157895,25.510526,19.473684,31.210526,23.0,1613,4.0,0.2,43.0,2.7,2.0,0.0,448,27.8,189,11.7,229,14.2,725,44.9,794.0,49.2,819.0,50.8,98.0,68,51,923.0,67.0,736.0,7.3,7.6,8.5,7.8,7.0,5.9,7.9,6.9,7.7,7.3,8.4,7.8,6.4,5.6,7.9,6.7,40.718725,-73.979426


# Filling in missing values  

First we will view the number of missing values, by column, in the `"combined"` dataframe.

In [38]:
# View the number of missing values for each column in "combined".
combined.isnull().sum()

DBN                                     0
SCHOOL NAME                             0
Num of SAT Test Takers                 28
SAT Critical Reading Avg. Score        28
SAT Math Avg. Score                    28
SAT Writing Avg. Score                 28
sat_score                              28
Total Cohort                           53
Total Grads - n                        71
Total Grads - % of cohort              71
Total Regents - n                      71
Total Regents - % of cohort            71
Total Regents - % of grads             72
Advanced Regents - n                   71
Advanced Regents - % of cohort         71
Advanced Regents - % of grads          72
Regents w/o Advanced - n               71
Regents w/o Advanced - % of cohort     71
Regents w/o Advanced - % of grads      72
Local - n                              71
Local - % of cohort                    71
Local - % of grads                     72
Still Enrolled - n                     71
Still Enrolled - % of cohort      

There are numerous missing values, exclusively in the quantitative columns, originating from either the original raw data or from the left join operations when we combined the dataframes.  

### The K-Nearest Neighbors algorithm

To fill in the missing data, we will use the K-Nearest Neighbors algorithm.  This algorithm estimates missing values by taking the average of the "K" nearest observed values in the same column as the missing data (the "nearest neighbors").  These "nearest neighbors" are the rows of data that are "closest" to the target row, based on calculating a distance metric (i.e. Euclidean distance) using the values in the other columns.  

The algorithm only works on quantitative data, so first we will separate `"combined"` into a dataframe named `"object_df"` (containing only `object` data types), and a dataframe named `"numeric_df"` (containing only `float` or `int` data types).  We will fill in the missing data in `"numeric_df"` using the K-Nearest Neighbors algorithm with K=5.  We will then re-join `"object_df"` with the completely filled numeric data and re-assign the resulting dataframe back to `"combined"`.  

Lastly, we will view the missing values for each column in `"combined"` to confirm that none remain.

In [39]:
from knnimpute.argpartition import knn_impute_with_argpartition as knn

# Separate "combined" into "object" and "numeric" only dataframes, fill
# the missing values in the "numeric" dataframe, re-join with the
# "object" dataframe, re-assign back to "combined" and view the missing
# values for each column.
object_df = combined.select_dtypes("object").copy()
numeric_df = combined.select_dtypes(["float64", "int64"]).copy()
filled_numeric_matrix = knn(numeric_df.values, numeric_df.isnull().values, k=5)
filled_numeric_df = pd.DataFrame(filled_numeric_matrix,
                                 columns=numeric_df.columns)
combined = object_df.join(filled_numeric_df)
combined.isnull().sum()

DBN                                   0
SCHOOL NAME                           0
Num of SAT Test Takers                0
SAT Critical Reading Avg. Score       0
SAT Math Avg. Score                   0
SAT Writing Avg. Score                0
sat_score                             0
Total Cohort                          0
Total Grads - n                       0
Total Grads - % of cohort             0
Total Regents - n                     0
Total Regents - % of cohort           0
Total Regents - % of grads            0
Advanced Regents - n                  0
Advanced Regents - % of cohort        0
Advanced Regents - % of grads         0
Regents w/o Advanced - n              0
Regents w/o Advanced - % of cohort    0
Regents w/o Advanced - % of grads     0
Local - n                             0
Local - % of cohort                   0
Local - % of grads                    0
Still Enrolled - n                    0
Still Enrolled - % of cohort          0
Dropped Out - n                       0


# Adding a school district column to `"combined"`  

It will be useful to be able to group the data by school district, particularly for plotting data on a geographic map.  As we discovered earlier, the first 2 digits of a school's `"DBN"` represent the school district it belongs to.  So we will add a new school district column, named `"school_dist"`, by taking the first 2 digits of each value in the `"DBN"` column.  We will then view the first 5 rows of `"DBN"` and `"school_dist"` to make sure the new column was created properly.

In [40]:
# Add a "school_dist" column and view the first 5 rows of "DBN" and
# "school_dist".
combined["school_dist"] = combined["DBN"].apply(lambda string: string[:2])
combined[["DBN", "school_dist"]].head()

,DBN,school_dist
0,01M292,01
1,01M448,01
2,01M450,01
3,01M509,01
4,01M539,01


# Exporting `"combined"` to a csv file  

We will now export the final cleaned `"combined"` dataframe to a csv file named `"nyc_hs_data_clean.csv"` in the `"data/processed"` folder of this project.

In [41]:
# Export the final cleaned "combined" dataframe to a csv file.
combined.to_csv("../data/processed/nyc_hs_data_clean.csv", index=False)

# Conclusion  

In this project our goal was to clean and consolidate several datasets relating to the SAT and NYC high schools.  We wanted to produce a final cleaned dataset that can be used in further analyses to explore if the SAT is unfair to particular groups of students.  

The raw data consisted of 8 separate datasets with a combined total of 65,875 rows and 3,861 columns.  We explored each of the datasets to understand them better and then cleaned them by correcting data errors, removing unnecessary columns, converting data types and filling in missing values.  The final result of this project is a single consolidated csv file containing 362 rows and 79 columns.  This final dataset is clean, consistent, relevant and ready to use for further analysis and modeling to explore the fairness of the SAT.  

In the [analyzing-the-sat](https://github.com/JustinToribio/analyzing-the-sat) project, which can also be found in this GitHub repository, we performed an exploratory and visual analysis of the dataset produced here (`"nyc_hs_data_clean.csv"`) to uncover any bias in the SAT.